# Hello, welcome to the FMR Python Automation tutorial!

## Context and Background

So what are we doing here? You're probably running the ferromagnetic resonance experiment conducted in the spin lab at the LUMS Department of Physics. See the experiment [here](https://physlab.org/story/ferromagnetic-resonance-for-investigating-magnetization-dynamics-in-magnetic-thin-films/). We'll be seeing how to run the instruments associated with this lab automatically with some Python code. There are three major instruments:

- The [SRS SR830](https://www.thinksrs.com/products/sr810830.htm) Lock-In Amplifier
- The [Kepco BOP 50-8D](https://www.kepcopower.com/bopmod.htm) Power Supply
- The [HP 8673G](https://www.testwall.com/product/agilent-hp-8673g/) (official website unavailable) Signal Generator

The [GitHub](https://github.com/sjshamsi/SpinLab_FMR_Automation) for this lab should have all these manuals.

Additionally, [here](https://www.youtube.com/watch?v=1HQxnz3P9P4) is a nice YouTube video detailing a lot of what we're about to discuss.


### The GPIB Standard

The [General Purpose Interface Bus](https://www.contec.com/support/basic-knowledge/daq-control/gpib-communication/) (GPIB) started off as HPIB (Hewlett-Packard Interface Bus) back when it was an internal interface at HP, way back in the 70s. So if you encounter an older instrument that says it uses HPIB (like our HP 8673G SG), don't worry, it's the same physical connector. The IEEE standardised it with a communications protocol called IEEE-488. Since then, the interface has been approved by the IEEE for standard use and a lot of lab instruments from different manufacturers now feature a GPIB connector. The new protocol enabling standardised use is called IEEE488.2 and it remains to be the modern standard today. This is relevant because our HP 8673G does not support standard IEEE-488.2 commands (but if you're just here to run the FMR experiment, you don't have to worry about this). Although modern lab instruments increasingly feature USB and ethernet connectors, GPIB is still alive and well in many labs today.

Physically, a GPIB cable usually features a thick wire and 24 pins at the connector. We're not going to get into what each pin is for. The connector comes in plug and receptacle (male and female) forms. The image below shows what a typical GPIB cable looks like.

<img src="https://www.meilhaus.de/cosmoshop/default/pix/a/g/1274088903-8796.2.jpg"  width="50%">

Looking at the cable from another angle, you'll notice that this particular kind of cable has two connectors per side of the cable, one plug and one receptacle.

<img src="https://il.farnell.com/productimages/standard/en_GB/1214526-40.jpg"  width="40%">

Why would this be? Turns out, GPIB has many cool tricks up its sleeve, including the possibility of cross-connecting multiple devices so that they all connect to just one port on your computer. This can be done in many ways. You could connect your instruments to a PC in a 'star' configuration:

<img src="https://www.contec.com/support/basic-knowledge/daq-control/gpib-communication/-/media/Contec/support/basic-knowledge/daq-control/gpib-communication/images/img_gpib-communication_07.jpg"  width="50%">

This is generally not recommended. A better solution is 'daisy-chaining', where you link all devices in 'series'. This also keeps things cleaner:

<img src="https://www.contec.com/support/basic-knowledge/daq-control/gpib-communication/-/media/Contec/support/basic-knowledge/daq-control/gpib-communication/images/img_gpib-communication_06.jpg"  width="50%">

Just don't create a loop at any point:

<img src="https://www.contec.com/support/basic-knowledge/daq-control/gpib-communication/-/media/Contec/support/basic-knowledge/daq-control/gpib-communication/images/img_gpib-communication_08.jpg"  width="50%">

So we daisy-chain our instruments to the connector in the PC, as shown in the following image:

<img src="media/foo.png"  width="25%">

This brings us to,

### Computer Interfaces

The GPIB connector may look unusual to you. Indeed, this port isn't common to many computers, and an interface solution must be employed. This might be a USB to GPIB interface like the Keithley KUSB-488B:

<img src="https://www.conrestestequipment.com/wp-content/uploads/2019/10/KEI-KUSB-488B.jpg"  width="50%">

Or the NI PCIe-GPIB+ controller (which is the one we're using):

<img src="https://www.meilhaus.de/cosmoshop/default/pix/a/g/1324374935-24837.2.jpg"  width="50%">

There are a couple of other manufacturers that make GPIB controllers. These controllers connect the PC to the instruments and allow us to communicate with them. Let us now talk about,

### Software Drivers

Where we have hardware (the GPIB interface) we need software to operate it. We have some options here.

NI and Keithley both maintain Windows software for VISA-based communication. VISA is the **Virtual Instrument Software Architecture**. The idea is that an experiment can require lab instruments with very different connectors. You might have an instrument with GPIB, one with just a USB port, and one with an RS-232 parallel connector. VISA makes it possible to communicate with instruments in the same manner, regardless of the physical connection.

Anyways, NI maintains **NI-VISA** and Keithley maintains the **Keithley I/O Layer**. NI-VISA is more popular/supported and is what we use. Among other things, the software:

1. Provides drivers to use the specific controller
2. Provides an API through which IEEE-488.1 and SCPI/IEEE-488.2 commands are sent to instruments under the VISA architecture

SCPI is just another layer over the IEEE-488.2 communications standard which standardises the syntax of what we sent to lab instruments. For instance, with SCPI, two voltmeters from different manufacturers might have the same command to make a measurement, `VOLT?`. Commands also are sent as ASCII strings which makes things more comprehensible as well. Note that completely standardised commands are not always necessary nor always possible. So generally, you'll be reading the manual for your respective instrument which should detail what SCPI commands to use for any action.

You can *generally* use one manufacturer's VISA software with another manufacturer's GPIB interface. For instance, NI-VISA will automatically detect Keithley KUSB-488A drivers and don't install their own IEEE-488 drivers, allowing you to use the NI-VISA API with the KUSB-488A. This isn't always true for every combination and software version though, and trial and error is necessary.

### Programming

Okay so we've physically connected the instruments via GPIB to our GPIB controller attached to our computer. We've installed NI-VISA, which automatically installs device drivers for the NI-PCIe-GPIB+ card as well as the API used to send commands. We also have read the instrument's manual to check which commands to send. Now what? Where do I type in these commands? How do I talk to my instruments? Where does Python come in?

Good questions! Firstly, when installing NI-VISA or Keithley I/O Layer, the software come with little programs which allow you to detect connected instruments and then send individual commands. They may also come with add-ons which allow programming environments like LabVIEW to interface with these instruments. But to use these software with Python, we need a module called `pyvisa`.

`pyvisa` acts as a Python wrapper for these software, allowing you to combine instrument communication with the flexibility and power of Python. You can learn more about `pyvisa` from its documentation [here](https://pyvisa.readthedocs.io/). `pyvisa` allows you to use the NI or Keithley APIs, but also has a native, python based API called `pyvisa-py`. This comes in the shape of a separate module which you can install. However, `pyvisa-py` isn't mature yet and has a lot of missing implementation. This can cause problems in many cases, for instance in use with older instruments. So we use the NI-VISA API with `pyvisa`. This is what our code is built upon.

## How-To/Setup

Okay so we've discussed the background to our programming implementation for the FMR lab. To summarise, if you'd like to recreate our setup, follow these steps:

- Install a GPIB interface to your computer (Windows PCs have more support)
- Daisy-chain your instruments via GPIB cables and connect them to your PC's GPIB interface
- Install the latest NI-VISA
- Install a Python implementation to your PC (Conda/Anaconda recommended)
- Create an environment for your experiment
- In your environment, install the following packages:
    * `pyvisa` (Python wrapper for VISA API)
    * `matplotlib` (We'll me making plots)
    * `ipympl` (Necessary for liveplots)
    * `pandas` (Needed for some internal functions. Plus good to have for your analyses!)
    * `jupyter` (or any other IDE you want)
- Download the files from the GitHub and save them where ever you like
- Navigate to the saved directory and create your scripts/notebooks within it so they can use the downloaded scripts

That's all!

## Our Code

We'll go over our code and how to use it.

**Structure**

Our code is based on just a couple of Python scripts. These are:
- `instrument_base.py`
- `bop50_8d.py`
- `hp_8673g.py`
- `srs_sr830.py`
- `fmr_experiment.py`

The first four files help us initialise and interact with our instruments. The last file `fmr_experiment.py` gives us some methods to then conduct various sweeps.

**What do I need to do to use these scripts?**

You'll need to make sure that whatever Python file you're using to do your analysis (be it a simple Python script or a Jupyter `.ipynb` notebook file) has those 5 files in the same directory. Of course you'll also want to make sure that the PC that you're running these files on is also connected with our instruments via GPIB.

For instance, we're using this file, `tutorial.ipynb` to run some example code. So we'll want to make sure that those 5 Python scripts are in the same folder as this file. Let us begin.

### Calling our Experiment Class

Our experiment has a class, which helps it gather all the resources it needs among other things. If you're unfamiliar with this, don't worry. Simply import the class like so:

In [1]:
from fmr_experiment import Experiment

In [2]:
# Now let's initialise the class
E = Experiment()

Welcome to the FMR Experiment!
Here are some default experiment parameters.

PS Output Current (A) :	 6.29627
PS Output Voltage (V) :	 47.53
PS Output Mode (Current/Voltage) :	 Constant Current
SG Frequency :	 FR03000000000HZ
SG RF Output :	 On
SG RF Output Level :	 LE-70.0DM
LIA Time Constant :	 0.3
LIA Sensivity :	 0.0002
Sensivity Delay (s) :	 3
Read Repetitions :	 1
Read Repetition Delay :	 0
Repetition Averaging Function :	 <function mean at 0x000001CE31F4BF40>
Read Delay :	 0.02
From 0 Delay (s) :	 4
Log File :	 C:\Users\physlab\Desktop\FMR_Python_Automation\SpinLab_FMR_Automation\Experiment_Logs\FMR_log_2023-6-23_13-0-46.log


Great! We've initialised an `Experiment` object and have named it `E`. Once initialised, a bunch of experiment parameters get printed out. Let's go over them:

- **PS Output Current (A)** shows the measured power supply current in Amperes. It's a bit out of wack because the set current is actually 0.
- **PS Output Voltage (V)** shows the measured power supply voltage in Volts. Same reason as the measurement current for why it isn't 0.
- **PS Output Mode (Current/Voltage)** shows whether the power supply is in constant current or constant voltage mode.
- **SG Frequency** is the RF output level from the signal generator.
- **SG RF Output** shows if the signal generator's RF output is on.
- **SG RF Output Level** shows the RF output's level in dB.
- **LIA Time Constant** shows the time constant for our lock-in amplifier in seconds.
- **LIA Sensitivity** shows the absolute largest signal it can measure in the current sensitivity setting. The sensitivity is in Volts or Amperes depending on the measurement setting.
- **Sensitivity Delay (s)** is the amount of time (in seconds) paused when the LIA sensitivity is changed. This is because some time is required after changing the sensitivity before stable reading can be achieved.
- **Read Repetitions** is the amount of readings taken per measurement of A and B. More readings give us a more stable, accurate result.
- **Read Repetition Delay** is the amount of time (in seconds) between reach read repetition.
- **Repetition Averaging Function** is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averaging than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **Read Delay** is the delay (in seconds) after the frequency or field is changed, before a measurement is taken.
- **From 0 Delay** is the delay (in seconds) after the frequency or field begins from 0 (occurs when initial parameters are being set), before a measurement is made.
- **Log File** is the path to the experiment log file which records every instrument communication. It's always a good idea to look at the log file for clues when things are acting up for unknown reasons.

These are just some of the experiment parameters which might be useful for you to take note of before your experiment. Of course, you can change most of these.

We will now import some packages we might use.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
%matplotlib qt # We need this for liveplots in jupyter
plt.style.use('seaborn') # Make our plots pretty!
plt.style.use('seaborn-colorblind') # Make our plots accessible!

UsageError: unrecognized arguments: # We need this for liveplots in jupyter


In [ ]:
# Changing any parameters we need to change
E.SG.level = '-3'

# Creating some constants to help us later
save_dir = r'C:\Users\plyslab\Desktop\different_ranges'

### Available Methods

Now that the experiment object is initialised, we can talk about the methods available. This will be easy, don't worry. We only have three methods to be aware of:

- `sweep_field`
- `sweep_frequency`
- `make2D`

Let's go over each method and see how to use it.

### `sweep_field`

As the name suggests, this method is used to conduct a magnetic field sweep at a particular output RF frequency. This method's arguments are:

- **frequency** *number*: Enter the frequency (in GHz) at which you want to conduct the magnetic field sweep.
- **fields** *number array*: Enter the magnetic field values (in Oe) through which you'd like to sweep in an array.
- **save_dir** *string*: Enter the path of the directory where you'd like to save your sweep data. If save directory does not exist, the program will create it (given that the parent directories exist).
- **livefig** *bool*: Would you like to show a live plot of the sweep as it happens? Pass along `True` or `False` (it's `True` by default).
- **savefig** *bool*: Would you like to save the sweep plot after it's done? Pass along `True` or `False` (it's `True` by default).
- **closefig** *bool*: Would you like to close the liveplot after it's done? Pass along `True` or `False` (it's `False` by default). There's usually no reason to automatically close the figure after the sweep is done, except if you're doing a lot of sweeps at the same time. In that case, closing the figures after the sweeps are done can save memory.
- **file_prefix** *string*: The sweep's savefiles usually look something like `'freq_3.0_GHz_field_80-120_Oe_-3_dB.csv'` by default. If you'd like to add a string before this default fileneme, you can pass it along to the `file_prefix` argument. For example, I can pass along the string `'NiFe_'` here if I'm investigating a NiFe sample to get a filename `'NiFe_freq_3.0_GHz_field_80-120_Oe_-3_dB.csv'`.
- **sen** *float*: Want to start your sweep with a particular sensitivity? Put it in here. The default value is $0.0002$ V.
- **sen_delay** *number*: How long (in seconds) do you want to wait before the LIA resumes taking readings after its sensitivity changes? By default this takes $3$ s. This delay is good to have as the LIA needs some time after sensitivity changes in order to stabilise.
- **read_reps** *int*: How many readings should the LIA take before making a measurement? This is $1$ by default. More repetitions usually help with unstable readings (especially at higher frequencies).
- **rep_delay** *number*: How long (in seconds) do you want to wait in between LIA readings? The default value is $0$ s.
- **read_delay** *number*: How long (in seconds) do you want to wait after the frequency or field is changed, before a measurement is taken? The default value is $0$ s.
- **from0delay** *number*: How long (in seconds) do you want to wait after the frequency or field begins from $0$ (occurs when initial parameters are being set), before a measurement is made? The default value is $4$ s.
- **avg_func** *function*: This is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averaging than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **return_XY** *bool*: Do you want the function to return two numpy arrays with the *X* and *Y* values respectively? If so, pass along `True` or `False`. The default value is `False`.

In [ ]:
# Let's do one sweep @ 3 GHz with our fields in a range from 0 - 170 Oe
frequency = 3
field_range = np.linspace(0, 170, 200)

E.sweep_field(3, field_range, save_dir=save_dir, savefig=True)

Great! The code cell above should eventually show us a graph like the following:

<img src="media/freq_3.0_GHz_field_0.0-170.0_Oe.png"  width="50%">


In [ ]:
# Looks good. Now let's do a sweep at 4.0 GHz with a field range of 70 - 270 Oe
frequency = 4.0
field_range = np.linspace(70, 270, 200)

E.sweep_field(frequency, field_range, save_dir=save_dir, savefig=True)

The code cell above should show something like:

<img src="media/freq_4.0_GHz_field_70.0-270.0_Oe.png"  width="50%">

### `sweep_frequency`

As the name suggests, this method is used to conduct a radio frequency sweep at a particular magnetic field. This method's arguments are:

- **field** *number*: Enter the field (in Oe) at which you want to conduct the frequency sweep.
- **frequencies** *number array*: Enter the frequency values (in GHz) through which you'd like to sweep in an array.
- **save_dir** *string*: Enter the path of the directory where you'd like to save your sweep data. If save directory does not exist, the program will create it (given that the parent directories exist).
- **livefig** *bool*: Would you like to show a live plot of the sweep as it happens? Pass along `True` or `False` (it's `True` by default).
- **savefig** *bool*: Would you like to save the sweep plot after it's done? Pass along `True` or `False` (it's `True` by default).
- **closefig** *bool*: Would you like to close the liveplot after it's done? Pass along `True` or `False` (it's `False` by default). There's usually no reason to automatically close the figure after the sweep is done, except if you're doing a lot of sweeps at the same time. In that case, closing the figures after the sweeps are done can save memory.
- **file_prefix** *string*: The sweep's savefiles usually look something like `'field_100_Oe_freq_3-10_GHz_-3_dB.csv'` by default. If you'd like to add a string before this default fileneme, you can pass it along to the `file_prefix` argument. For example, I can pass along the string `'NiFe_'` here if I'm investigating a NiFe sample to get a filename `'NiFe_field_100_Oe_freq_3-10_GHz_-3_dB.csv'`.
- **sen** *float*: Want to start your sweep with a particular sensitivity? Put it in here. The default value is $0.0002$ V.
- **sen_delay** *number*: How long (in seconds) do you want to wait before the LIA resumes taking readings after its sensitivity changes? By default this takes $3$ s. This delay is good to have as the LIA needs some time after sensitivity changes in order to stabilise.
- **read_reps** *int*: How many readings should the LIA take before making a measurement? This is $1$ by default. More repetitions usually help with unstable readings (especially at higher frequencies).
- **rep_delay** *number*: How long (in seconds) do you want to wait in between LIA readings? The default value is $0$ s.
- **read_delay** *number*: How long (in seconds) do you want to wait after the frequency or field is changed, before a measurement is taken? The default value is $0$ s.
- **from0delay** *number*: How long (in seconds) do you want to wait after the frequency or field begins from $0$ (occurs when initial parameters are being set), before a measurement is made? The default value is $4$ s.
- **avg_func** *function*: This is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averaging than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **return_XY** *bool*: Do you want the function to return 2 numpy arrays with the *X* and *Y* values respectively? If so, pass along `True` or `False`. The default value is `False`.

In [ ]:
# Looking at the field sweep (0 - 170 Oe) conducted at 3 GHz, we can see that the midpoint seems to be at 99.1 Oe.
# This is a great opportunity to conduct a frequency sweep (say, from 2.5 - 3.5 GHz @ 99.1 Oe) to see if the midpoint lands at 3.0 GHz.

field = 99.1
freq_range = np.linspace(2.5, 3.5, 200)

E.sweep_frequency(field, freq_range, save_dir=save_dir, savefig=True)

The cell above shoudl get us soemthing like:

<img src="media/field_99.1_Oe_freq_2.5-3.5_GHz.png"  width="50%">

To address the obvious: yes, while we do see the expected curves, they're a bit... wobbly. Additionally, the midpoint doesn't exactly seem to be at 3.0 GHz, but this can be because of the wobbles (imagine a positive voltage effect on Channel X and a negative one on Channel Y @ 3.0 GHz) I do not know why, and if you have any ideas, please let me know.

### `make2D`

Would you like to make a 2D sweep of power absorption along the frequency and field axes? This method's arguments are:


- **frequencies** *number array*: Enter the frequency values (in GHz) through which you'd like to sweep in an array.
- **fields** *number array*: Enter the magnetic field values (in Oe) through which you'd like to sweep in an array.
- **save_dir** *string*: Enter the path of the directory where you'd like to save your sweep data. If save directory does not exist, the program will create it (given that the parent directories exist).
- **primary** *string*: Whenever we conduct a 2D sweep, the sweep needs to select one of the parameters (frequency or field) to be the 'primary' parameter with the other being the 'secondary' parameter. Pass along either `'frequency'` or `'field'` in this argument to decide the primary parameter. The default is `'frequency'`.
- **channel** *string*: both *X* and *Y* channel measurements from the LIA can show power absorption. Which channel do you want to use in the 2D sweep? Pass along `'X'` or `'Y'` in the argument here to decide. The default is `'X'`.
- **livefig** *bool*: Would you like to show a live plot of an individual sweep as it happens? Pass along `True` or `False` (it's `False` by default). Note that this is different from the 2D liveplot itself, which is always shown.
- **savefig** *bool*: Would you like to save each individual sweep plot after it's done? Pass along `True` or `False` (it's `False` by default). Note that this is different from the 2D liveplot itself, which is always saved.
- **closefig** *bool*: Would you like to close each individual sweep liveplot after it's done? Pass along `True` or `False` (it's `False` by default). If you're doing a lot of sweeps at the same time (which tends to happen during a 2D sweep), closing the figures after the sweeps are done can save memory. Note that this is different from the 2D liveplot itself.
- **file_prefix** *string*: The sweep's savefiles usually look something like `'2Dsweep_freq_3-10_GHz_field_100-500_Oe_-3_dB_channel_X_Unintegrated.npy'` by default. If you'd like to add a string before this default fileneme, you can pass it along to the `file_prefix` argument. For example, I can pass along the string `'NiFe_'` here if I'm investigating a NiFe sample to get a filename `'NiFe_2Dsweep_freq_3-10_GHz_field_100-500_Oe_-3_dB_channel_X_Unintegrated.npy'`.
- **sen** *float*: Want to start your sweeps with a particular sensitivity? Put it in here. The default value is $0.0002$ V. Note that if the sensitivity changes during one individual sweep, it resets to this value before the next individual sweep.
- **sen_delay** *number*: How long (in seconds) do you want to wait before the LIA resumes taking readings after its sensitivity changes? By default this takes $3$ s. This delay is good to have as the LIA needs some time after sensitivity changes in order to stabilise.
- **read_reps** *int*: How many readings should the LIA take before making a measurement? This is $1$ by default. More repetitions usually help with unstable readings (especially at higher frequencies).
- **rep_delay** *number*: How long (in seconds) do you want to wait in between LIA readings? The default value is $0$ s.
- **read_delay** *number*: How long (in seconds) do you want to wait after the frequency or field is changed, before a measurement is taken? The default value is $0$ s.
- **from0delay** *number*: How long (in seconds) do you want to wait after the frequency or field begins from $0$ (occurs when initial parameters are being set), before a measurement is made? The default value is $4$ s.
- **avg_func** *function*: This is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averagings than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **integrate** *bool*: Do you want the *X* or *Y* channel sweep to be integrated? If so, pass along `True` or `False`. The default value is `False`. Note that this doesn't work very well for unstable power absorption values.

In [ ]:
%%time

#define the frequency range
freq_step = 0.01 # 10  MHz
freq_range = np.arange(2, 4 + freq_step, freq_step)

#define field range
field_step = 10
field_range = np.arange(0, 400 + field_step, field_step)

E.SG.level = '-3'

save_dir = r'C:\Users\physlab\Desktop\FMR_Python_Automation\Data\NiFe_2_2d_plot2'
arr = E.make_2D(freq_range,
          field_range,
          save_dir,
          primary='frequency',
          channel='X',
          integrate=False,
          read_delay=0.05,
          read_reps=1,
          rep_delay=0,
          avg_func=np.average)

The code above just conducts multiple sweeps and adds them to a 2D array before displaying them. You should end up with something like:

<img src="/media/freq_2.0-5.0_field_-200-200_Oe.png" width="50%" />